# Résumé

L'objectif de ce notebook est de faire du "Document Classification", c'est une sous-partie du NLP. Pour cela, nous prenons les données qui sont [ici](https://api.github.com/repos/Microsoft/vscode/issues). Les données peuvent être récupérées soit via l'API [PyGithub](https://github.com/PyGithub/PyGithub), soit directement avec la commande curl. Nous devrons ensuite classer les différentes "issues" sous les différents labels (bug, feature-request). Pour finir, nous fournirons une méthode qui prendra en paramètre, un titre et un corps de texte et qui labellisera cette nouvelle entrée.

Plan
========
1. Construction du dataset
2. Séparation des datasets
3. Création du modèle
3. Utilisation du classifier

In [1]:
#Diférents imports utilisés par la suite
import pandas as pd #Gestion des dataframes
import nltk # Traitement du langage naturel
import numpy as np
#Apprentissage
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from pprint import pprint
from time import time
import logging

In [2]:
#Constants utilisées par la suite
LABEL_FQ = 'feature-request'
LABEL_BUG = 'bug'
LABEL_OTHER = 'other'
LABELS = [LABEL_BUG, LABEL_FQ, LABEL_OTHER]

# Construction du dataset

Pour cet exercice, on ne prendra que le titre, le corps et le label de l'issue. On va faire 3 classes différentes, 'bug', 'feature-request', 'other'. On considère que chaque input n'a qu'un seul label.

In [3]:
#Fonction qui permet de redéfinir les autres labels que bug et feature-request à other
def filter_label(labels):
    
    if LABEL_FQ  in labels:
        return LABEL_FQ
    elif LABEL_BUG in labels:
        return LABEL_BUG
    
    return LABEL_OTHER

In [4]:
issues = pd.read_csv('./issues.csv') #importation des données téléchargées au préalable
issues = issues.loc[:,['title','body','labels']] #On conserve seulement titre, body et labels
issues.head()

,title,body,labels
0,Panel badge is an odd shape when a single digit,Need to update the css so that this badge beco...,[]
1,custom titlebar : fullscreen very top dragging...,- VSCode Version: Insiders 1.26\r\n- OS Versio...,[]
2,Localized descriptions for built-in extensions...,Fixes #54111,[]
3,editor automatically removing characters from ...,Issue Type: <b>Bug</b>\r\n\r\nthe editor is re...,[]
4,[js] Add auto completion for computed property...,Currently intellisense doesn't work for comput...,"[{'id': 291124272, 'node_id': 'MDU6TGFiZWwyOTE..."


In [5]:
#Transformation des labels. On ne garde qu'un seul label (bug, feature-request et other)
for ind in issues.index:
    row = issues.loc[ind]
    labels = eval(row['labels'])
    tmp = []

    if len(labels) > 0: # S'il y a au moins un label, 3 possibilitées d'affectation
        for l in labels:
            tmp.append(l['name'])

        new_label = filter_label(tmp)
        issues.loc[ind, 'labels'] = new_label
    else:
        issues.loc[ind, 'labels'] = LABEL_OTHER #Sinon c'est other

In [6]:
issues.head()

,title,body,labels
0,Panel badge is an odd shape when a single digit,Need to update the css so that this badge beco...,other
1,custom titlebar : fullscreen very top dragging...,- VSCode Version: Insiders 1.26\r\n- OS Versio...,other
2,Localized descriptions for built-in extensions...,Fixes #54111,other
3,editor automatically removing characters from ...,Issue Type: <b>Bug</b>\r\n\r\nthe editor is re...,other
4,[js] Add auto completion for computed property...,Currently intellisense doesn't work for comput...,other


In [7]:
print(issues.labels.value_counts())
print('Totale : {}'.format(issues.shape[0]))

feature-request    2833
other              1644
bug                 904
Name: labels, dtype: int64
Totale : 5381


# Séparation des datasets

Dans cette section, nous séparons les données en 2 datasets. Ceci afin de valider le classifieur. Nous allons avoir un dataset pour l'entrainement et un pour le test. On garde 70% de chaque classe pour l'entraînement, et le reste de 30%.

In [8]:
dfTrain = {}
dfTest = {}
for l in LABELS:
    dfTrain[l] = issues[issues.labels == l].sample(frac=0.7)
    dfTest[l] = issues[issues.labels == l].drop(dfTrain[l].index)

dfTrain = pd.concat([dfTrain[l] for l in LABELS ], axis=0)
dfTest = pd.concat([dfTest[l] for l in LABELS ], axis=0)

In [9]:
dfTrain.head()

,title,body,labels
4865,Source control integration ignores files stage...,- VSCode Version: Code 1.19.3 (7c4205b5c6e52a5...,bug
3420,Consider using a lighter blue for menu selection,The current blue is VSCode blue which is too s...,bug
3771,Diff editor: Git actions are enabled when doin...,- VSCode Version: 1.22.2\r\n- OS Version: Wind...,bug
1475,Typescript symbols aren't displayed when using...,- VSCode Version: Code - Insiders 1.11.0-insid...,bug
4856,"searching for ""IntelliSense"" or ""completions"" ...",I had someone ask how to turn off completions ...,bug


In [10]:
dfTrain.labels.value_counts()

feature-request    1983
other              1151
bug                 633
Name: labels, dtype: int64

In [11]:
dfTest.head()

,title,body,labels
68,Windows update failed: Access is denied (resou...,\r\nJul 14 17:24:09.411 INFO Starting: C:\Prog...,bug
81,Toggle Word Wrap doesn't work with the custom ...,Issue Type: <b>Bug</b>\r\n\r\n(Using Windows 1...,bug
85,Toggling sidebar switches away from settings e...,Issue Type: <b>Bug</b>\r\n\r\n**Repo**\r\n1. O...,bug
100,Extensions Debug breakpoint don't work,- VSCode Version: Code 1.18.0 (dcee2202709a4f2...,bug
127,Terminal: can't call `#sendText` with a long text,- VSCode Version: Code 1.18.0\r\n- OS Version:...,bug


In [12]:
dfTest.labels.value_counts()

feature-request    850
other              493
bug                271
Name: labels, dtype: int64

In [13]:
dfTrain = dfTrain[dfTrain.labels != LABEL_OTHER]
dfTest = dfTest[dfTest.labels != LABEL_OTHER]

In [14]:
dfTrain.labels.value_counts()

feature-request    1983
bug                 633
Name: labels, dtype: int64

In [15]:
#On mélange les données
dfTrain = shuffle(dfTrain)
dfTest = shuffle(dfTest)

On constate un déséquilibre au niveau  du nombre d'éléments par classe. Cela pourra poser des difficultés pour l'apprentissage.

# Création du modèle

Maintenant que nous avons nos datasets, nous allons traiter nos données, afin d'aider notre classifier à trouver du sens. Pour ce faire, nous allons "stemmatiser" les différents textes, "tokanizer" pour récupérer les différents termes utilisés. Pour finir, notre dataset ressemblera à un Bag of Words (BoW). Concrètement nous aurons une matrice (n exemples x m mots). Les m mots sont tous les mots rencontrés dans le dataset d'entraînement. Les différentes valeurs correspondront au nombre de fois que le mot est utilisé par un exemple.

On arrange les données pour mélanger le titre avec le contenu

In [18]:
corpus = [str(row['title']) + ' ' + str(row['body']) for ind, row in dfTrain.iterrows()]
corpusTest = [str(row['title']) + ' ' + str(row['body']) for ind, row in dfTest.iterrows()]

CountVectorizer va créer notre Bag of Words, TfidTransfomer va pondérer et donner plus de valeurs aux mots fréquemment utilisés.
SGDC (Stochastic Gradient Descent Classifier) est une méthode d'apprentissage.

In [19]:
tokenizer = nltk.tokenize.TweetTokenizer()
pipeline =Pipeline([
    ('vect', CountVectorizer(stop_words='english', tokenizer=tokenizer.tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(class_weight='balanced')),
])

In [20]:
parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    'vect__ngram_range': ((1, 1), (1, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    #'clf__n_iter': (10, 50, 80),
}

In [ ]:
grid_searchSGD = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_searchSGD.fit(corpus, dfTrain.labels.values)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_searchSGD.best_score_)
print("Best parameters set:")
best_parameters = grid_searchSGD.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [22]:
clf = grid_searchSGD.best_estimator_

In [23]:
pred = clf.predict(corpusTest)
print(accuracy_score(y_pred=pred, y_true=dfTest.labels.values))
print(confusion_matrix(y_pred=pred, y_true=dfTest.labels.values))

0.8144513826940232
[[131 140]
 [ 68 782]]


On a une précision de 0.81%. La matrice de confusion (ou contingence) montre la répartition des prédictions. Si on a 100% de prédictions, on a une belle diagonale. On peut tester notre classifier sur de nouvelles données :

In [24]:
clf.predict(['Found a bug, please correct it quicly ! I can\'t work now'])[0]

'bug'

In [25]:
clf.predict(['Need a new feature to improve the software'])[0]

'feature-request'

Le classifier ne connait que 2 classes : bug et feature-request.

In [26]:
clf.predict(['Brian is in the kitchen'])[0]

'feature-request'

On rajoute le "Stemmer" pour améliorer la précision

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english", ignore_stopwords=True)
class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        analyzer = super(StemmedCountVectorizer, self).build_analyzer()
        return lambda doc: ([stemmer.stem(w) for w in analyzer(doc)])
stemmed_count_vect = StemmedCountVectorizer(stop_words='english')

pipeline =Pipeline([
    ('vect', stemmed_count_vect),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(class_weight='balanced')),
])

grid_searchSGDStem = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)
grid_searchSGDStem.fit(corpus, dfTrain.labels.values)

In [28]:
clf = grid_searchSGDStem.best_estimator_
pred = clf.predict(corpusTest)
print(accuracy_score(y_pred=pred, y_true=dfTest.labels.values))
print(confusion_matrix(y_pred=pred, y_true=dfTest.labels.values))
print(clf.predict(['Found a bug, please correct it quicly ! It\'s a bug'])[0])
print(clf.predict(['Need a new feature to improve the software'])[0])
print(clf.predict(['Brian is in the kitchen'])[0])

0.775200713648528
[[234  37]
 [215 635]]
bug
feature-request
bug


La précision est moins bonne. Cependant, d'après la matrice de confusion, le concept de bug a été mieux appris.

# Utilisation du classifier

On définit une classe IssueClassifier. Cette classe pourra classer une issue en fonction de son titre et de son contenu. Les classes possibles sont 'Bug' et 'feature-request'

In [29]:
class IssueClassifier:
    def __init__(self,clf):
        self.clf = clf
        
    def predict_issue(self, title, body):
        return self.clf.predict([title + ' ' + body])[0]

In [30]:
issueClf = IssueClassifier(clf)

In [31]:
issueClf.predict_issue(title = "Bug", body = "Hi, I found a bug ! Could you please correct it quickly ? It's a bug, bug, bug, bug")

'bug'

In [32]:
issueClf.predict_issue(title = "A Bug in the matrix", body = "Hi, I found a bug ! Steps to reproduce 1 step1 . Could you please correct it quickly ? Regards")

'bug'

In [33]:
issueClf.predict_issue(title = "Feature", body = "Hi, I need a new feature concerning the autocompletion")

'feature-request'

In [34]:
issueClf.predict_issue(title="Hell World !", body="Brian is in the kitchen")

'feature-request'

# Conclusion

Nous avons désormais un classifier d'issue basé uniquement sur le langage naturel (titre et corps). Le classifier est plutôt simple. Pour l'améliorer, on peut utiliser plus de données (seules les issues ouvertes ont été utilisées). De même, la représentation des données (BoW), n'est pas sans faille. On pourrait aussi utiliser du deep learning (DNN et LSTM)